In [1164]:
from pandas import DataFrame
#from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, precision_score, recall_score
#from sklearn import tree
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
#from sklearn.tree import export_text

from datetime import datetime
from datetime import timedelta

In [1165]:
df = pd.read_csv('/data/DEvideos.csv',
    low_memory=False)

df['trending_date'] = df.apply(lambda row: datetime.strptime(row['trending_date'], '%y.%d.%m'), axis=1)
df['publish_time'] = df.apply(lambda row: datetime.strptime(row['publish_time'], '%Y-%m-%dT%H:%M:%S.000Z'), axis=1)
df['days_until_trending'] = df.apply(lambda row: ((row['trending_date'] - row['publish_time']).days + 1), axis=1)

df['tags_count'] = df.apply(lambda row: len(row['tags'].split('|')), axis=1)
df['publish_hour'] = df['publish_time'].map(lambda x: x.hour)
df['publish_month'] = df['publish_time'].map(lambda x: x.month)
df['publish_year'] = df['publish_time'].map(lambda x: x.year)
df['publish_day_of_month'] = df['publish_time'].map(lambda x: x.day)
df['publish_weekday'] = df['publish_time'].map(lambda x: x.weekday()) # 0: Monday, 6: Sunday
df['trending_weekday'] = df['trending_date'].map(lambda x: x.weekday()) # 0: Monday, 6: Sunday

df['like_dislike_ratio'] = df.apply(lambda row: row['likes'] / (row['dislikes'] + 1), axis=1)
df['like_view_ratio'] = df.apply(lambda row: row['likes'] / (row['views'] + 1), axis=1)

df['ratings'] = df['likes'] + df['dislikes']
df['likes_per_rating'] = df['likes'] / df['ratings']
df['ratings_per_view'] = df['ratings'] / df['views']
df['comments_per_view'] = df['comment_count'] / df['views']

def assign_target_category(row):
    if row['days_until_trending'] == 0: 
        return '0 Days'
    elif row['days_until_trending'] == 1:
        return '1 Day'
    elif row['days_until_trending'] == 2:
        return '2 Days'
    elif row['days_until_trending'] <= 5:
        return '3 - 5 Days'
    else:
        return '> 5 Days'

df['target_category'] = df.apply(assign_target_category, axis=1)

N = len(df)
dropColumns = ['video_id', 'title', 'tags', 'thumbnail_link', 'description']
for column in df.columns:
    numberOfUniqueValues = df[column].nunique()
    if numberOfUniqueValues < 2:
        dropColumns.append(column)
    elif df[column].dtype == 'object' and numberOfUniqueValues > N * 0.9:
        dropColumns.append(column)
    elif df[column].isna().sum() / N > 0.95:
        dropColumns.append(column)
        
df.drop(columns=dropColumns, inplace=True)

df['channel_title'] = df['channel_title'].astype('category')
df['target_category'] = df['target_category'].astype('category')

In [1166]:
df.head(10)

,trending_date,channel_title,category_id,publish_time,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,...,publish_day_of_month,publish_weekday,trending_weekday,like_dislike_ratio,like_view_ratio,ratings,likes_per_rating,ratings_per_view,comments_per_view,target_category
0,2017-11-14,inscope21,24,2017-11-13 17:08:49,252786,35885,230,1539,False,False,...,13,0,1,155.346320,0.141957,36115,0.993631,0.142868,0.006088,1 Day
1,2017-11-14,LUKE! Die Woche und ich,23,2017-11-12 22:30:01,797196,53576,302,1278,False,False,...,12,6,1,176.818482,0.067205,53878,0.994395,0.067584,0.001603,2 Days
2,2017-11-14,LastWeekTonight,24,2017-11-13 07:30:00,2418783,97190,6146,12703,False,False,...,13,0,1,15.810965,0.040181,103336,0.940524,0.042722,0.005252,1 Day
3,2017-11-14,100SekundenPhysik,27,2017-11-12 15:00:01,380247,31821,458,1955,False,False,...,12,6,1,69.326797,0.083685,32279,0.985811,0.084890,0.005141,2 Days
4,2017-11-14,rezo,24,2017-11-12 13:10:36,822213,100684,2467,10244,False,False,...,12,6,1,40.795786,0.122455,103151,0.976084,0.125455,0.012459,2 Days
5,2017-11-14,Die Allestester,22,2017-11-13 01:49:24,32709,3093,296,63,False,False,...,13,0,1,10.414141,0.094558,3389,0.912659,0.103611,0.001926,1 Day
6,2017-11-14,Bodyformus,23,2017-11-13 11:59:04,308683,35704,578,1398,False,False,...,13,0,1,61.664940,0.115665,36282,0.984069,0.117538,0.004529,1 Day
7,2017-11-14,Jay & Arya,22,2017-11-13 18:09:55,181660,17998,169,554,False,False,...,13,0,1,105.870588,0.099075,18167,0.990697,0.100006,0.003050,1 Day
8,2017-11-14,TeddyComedy,23,2017-11-12 11:53:45,369173,16953,570,611,False,False,...,12,6,1,29.690018,0.045921,17523,0.967471,0.047466,0.001655,2 Days
9,2017-11-14,WALULIS,1,2017-11-13 15:06:12,62418,4749,44,425,False,False,...,13,0,1,105.533333,0.076083,4793,0.990820,0.076789,0.006809,1 Day


In [1256]:
#x = df.iloc[:, [1, 2]].values

x_df = DataFrame(index=df.index)
#features = ['channel_title'] # 60.0 / 61.77
#features = ['channel_title', 'comments_per_view'] # 61.31 / 62.22
#features = ['channel_title', 'publish_hour', 'publish_weekday'] # 60.02 / 61.75
#features = ['publish_day_of_month'] # 62.29
#features = ['publish_year'] # 62.41
#features = ['publish_day_of_month', 'publish_year'] # 62.75
#features = ['publish_month'] # 62.41
#features = ['publish_day_of_month', 'publish_month'] # 62.41
#features = ['publish_month', 'publish_year'] # 62.52
#features = ['publish_day_of_month', 'publish_year', 'publish_month'] # 62.81
#features = ['publish_time'] # 62.50
#features = ['publish_weekday'] # 62.37
#features = ['publish_hour'] # 62.98
#features = ['publish_weekday', 'publish_hour'] # 62.92
#features = ['publish_weekday', 'publish_time'] # 62.41
#features = ['publish_weekday', 'publish_year'] # 62.52
features = ['publish_weekday', 'publish_month'] # 63.27
#features = ['publish_weekday', 'publish_day_of_month'] # 62.26
#features = ['publish_weekday', 'publish_month', 'publish_year'] # 62.76
#features = ['publish_weekday', 'publish_day_of_month', 'publish_month', 'publish_year'] # 62.52
#features = ['publish_hour', 'publish_year'] # 62.68
#features = ['publish_hour', 'publish_month'] # 62.86
#features = ['publish_hour', 'publish_month', 'publish_year'] # 62.35
#features = ['publish_hour', 'publish_day_of_month'] # 62.63
#features = ['publish_hour', 'publish_day_of_month', 'publish_month'] # 62.56
#features = ['publish_hour', 'publish_day_of_month', 'publish_year'] # 62.65
#features = ['publish_hour', 'publish_day_of_month', 'publish_month', 'publish_year'] # 62.94
#features = ['publish_hour', 'publish_weekday', 'publish_month'] # 63.27
#features = ['publish_weekday', 'publish_month', 'channel_title'] # 62.59
for feature in features:
    feature_data = df[feature].astype(str)
    if df[feature].dtype.name in ['category', 'datetime64[ns]', 'bool']:
        x_label_encoder = preprocessing.LabelEncoder()
        x_label_encoder.fit(feature_data)
        x_df[feature] = x_label_encoder.transform(feature_data)
    else:
        x_df[feature] = feature_data

x = np.reshape(x_df, (-1, len(x_df.columns)))


In [1257]:
#y = df.iloc[:, -1].values
target = df['target_category'].astype(str)
y_label_encoder = preprocessing.LabelEncoder()
y_label_encoder.fit(target)
y = y_label_encoder.transform(target)

In [1258]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.40)

In [1259]:
#scaler = preprocessing.LabelEncoder()
#scaler.fit(x_train)
#x_train = scaler.transform(x_train)
#x_test = scaler.transform(x_test)

In [1260]:
classifier = KNeighborsClassifier(n_neighbors = 1000)
classifier.fit(x_train, y_train)

KNeighborsClassifier(n_neighbors=1000)

In [1261]:
y_pred = classifier.predict(x_test)

In [1262]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='micro')
recall = recall_score(y_test, y_pred, average='micro')

In [1263]:
print('Accuracy:')
print(accuracy)

print('Precision:')
print(precision)

print('Recall:')
print(recall)

Accuracy:
0.6274485798237023
Precision:
0.6274485798237023
Recall:
0.6274485798237023


In [1264]:
len(df[df['target_category'] == '1 Day']) / len(df)

0.6237512242899118